In [7]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [9]:
# --- carregar base
df_sim = pd.read_csv('./base_limpa/base_sim_dofet_limpa_remocao.csv')

In [10]:
df_sim['cat_peso_calc_2'] = ['PIG' if i=='PIG' else 'NAO_PIG' for i in df_sim['cat_peso_calc']]

In [13]:
df_sim_agreg_regiao = pd.DataFrame(
    df_sim.groupby(['cat_periodo_nasc', 'cat_peso_calc_2', 'ocor_REGIAO']).agg(Tamanho=('data_obito', 'size'))
).reset_index()

In [15]:
df_sim_agreg_brasil = pd.DataFrame(
    df_sim.groupby(['cat_periodo_nasc', 'cat_peso_calc_2']).agg(Tamanho=('data_obito', 'size'))
).reset_index()

In [17]:
df_sinasc = pd.read_csv('./base_limpa/base_sinasc_limpa_remocao.csv')

In [18]:
df_sinasc['cat_peso_calc_2'] = ['PIG' if i=='PIG' else 'NAO_PIG' for i in df_sinasc['cat_peso_calc']]

In [22]:
df_sinasc_agreg_regiao = pd.DataFrame(
    df_sinasc.groupby(['cat_periodo_nasc', 'cat_peso_calc_2', 'nasc_REGIAO']).agg(Tamanho=('data_nasc', 'size'))
).reset_index()

In [23]:
df_sinasc_agreg_brasil = pd.DataFrame(
    df_sinasc.groupby(['cat_periodo_nasc', 'cat_peso_calc_2']).agg(Tamanho=('data_nasc', 'size'))
).reset_index()

In [24]:
df_sinasc_agreg_regiao

,cat_periodo_nasc,cat_peso_calc_2,nasc_REGIAO,Tamanho
0,pre_termo,NAO_PIG,Centro-Oeste,209858
1,pre_termo,NAO_PIG,Nordeste,726299
2,pre_termo,NAO_PIG,Norte,310185
3,pre_termo,NAO_PIG,Sudeste,949222
4,pre_termo,NAO_PIG,Sul,331925
5,pre_termo,PIG,Centro-Oeste,50540
6,pre_termo,PIG,Nordeste,156865
7,pre_termo,PIG,Norte,52302
8,pre_termo,PIG,Sudeste,284784
9,pre_termo,PIG,Sul,92281


In [28]:
# ------------------------------------------------------------
# 0) AJUSTES E MAPAS
# ------------------------------------------------------------

# Mapa opcional para nomes "bonitos" das 4 combinações usuais
pretty_map = {
    ('pre_termo', 'PIG'): 'Preterm and SGA',
    ('pre_termo', 'NAO_PIG'): 'Preterm non-SGA',
    ('termo', 'PIG'): 'Term and SGA',
    ('termo', 'NAO_PIG'): 'Term and non-SGA'
}

def label_cat(row):
    t = (row['cat_periodo_nasc'], row['cat_peso_calc_2'])
    return pretty_map.get(t, f"{row['cat_periodo_nasc']} & {row['cat_peso_calc_2']}")

In [30]:
# ------------------------------------------------------------
# 1) MISSING VALUES (a partir das bases não agregadas)
#    -> altere os nomes se necessário
# ------------------------------------------------------------
def summarize_missing(df, vars_):
    out = df[vars_].isna().sum().to_frame('n_missing')
    out['perc_missing'] = 100 * out['n_missing'] / len(df)
    return out

miss_sinasc = summarize_missing(
    df_sinasc,
    ['cat_periodo_nasc', 'cat_peso_calc_2', 'nasc_REGIAO']
)
miss_sim = summarize_missing(
    df_sim,
    ['cat_periodo_nasc', 'cat_peso_calc_2', 'ocor_REGIAO']
)

print("Missing SINASC (variáveis usadas):")
print(miss_sinasc)
print("\nMissing SIM (variáveis usadas):")
print(miss_sim)


Missing SINASC (variáveis usadas):
                  n_missing  perc_missing
cat_periodo_nasc          0           0.0
cat_peso_calc_2           0           0.0
nasc_REGIAO               0           0.0

Missing SIM (variáveis usadas):
                  n_missing  perc_missing
cat_periodo_nasc          0           0.0
cat_peso_calc_2           0           0.0
ocor_REGIAO               0           0.0


In [32]:
# ------------------------------------------------------------
# 2) PREPARA CONTAGENS DO SINASC (REGIÃO E BRASIL)
# ------------------------------------------------------------
# Região: contagem por categoria + total da região
sinasc_reg = df_sinasc_agreg_regiao.copy()
sinasc_reg['SVN_cat'] = sinasc_reg.apply(label_cat, axis=1)

# total por REGIÃO (denominador)
denom_reg = (
    sinasc_reg.groupby('nasc_REGIAO')['Tamanho']
    .sum()
    .rename('n_total_reg')
    .reset_index()
)

# contagem por REGIÃO x CATEGORIA (numerador)
num_reg = (
    sinasc_reg
    .groupby(['nasc_REGIAO', 'SVN_cat'])['Tamanho']
    .sum()
    .rename('n_cat_reg')
    .reset_index()
)

# Brasil: contagem por categoria e total
sinasc_br = df_sinasc_agreg_brasil.copy()
sinasc_br['SVN_cat'] = sinasc_br.apply(label_cat, axis=1)

n_total_br = sinasc_br['Tamanho'].sum()
num_br = (
    sinasc_br.groupby('SVN_cat')['Tamanho']
    .sum()
    .rename('n_cat_br')
    .reset_index()
)
num_br['n_total_br'] = n_total_br
num_br['p_br'] = num_br['n_cat_br'] / num_br['n_total_br']

In [34]:
num_reg

,nasc_REGIAO,SVN_cat,n_cat_reg
0,Centro-Oeste,Preterm and SGA,50540
1,Centro-Oeste,Preterm non-SGA,209858
2,Centro-Oeste,Term and SGA,354192
3,Centro-Oeste,Term and non-SGA,1732884
4,Nordeste,Preterm and SGA,156865
5,Nordeste,Preterm non-SGA,726299
6,Nordeste,Term and SGA,1193753
7,Nordeste,Term and non-SGA,5637252
8,Norte,Preterm and SGA,52302
9,Norte,Preterm non-SGA,310185


In [36]:
# ------------------------------------------------------------
# 3) FUNÇÕES AUXILIARES
# ------------------------------------------------------------
def prev_ci_wald(n_cat, n_total):
    """Retorna string 'xx.xx (li-hi)' para % por 100 NV com IC95% (Wald)."""
    if n_total == 0 or pd.isna(n_total):
        return np.nan
    p = n_cat / n_total
    se = np.sqrt(p * (1 - p) / n_total) if n_total > 0 else np.nan
    lo = max(p - 1.96 * se, 0)
    hi = min(p + 1.96 * se, 1)
    return f"{p*100:.2f} ({lo*100:.2f}-{hi*100:.2f})"

def pvalue_reg_vs_br(n_cat_reg, n_total_reg, p_br):
    """Teste z bicaudal H0: p_reg = p_br (comparação região x Brasil)."""
    if n_total_reg == 0 or pd.isna(n_total_reg):
        return np.nan
    p_reg = n_cat_reg / n_total_reg
    se_h0 = np.sqrt(p_br * (1 - p_br) / n_total_reg)
    if se_h0 == 0:
        return np.nan
    z = (p_reg - p_br) / se_h0
    return 2 * (1 - norm.cdf(abs(z)))


In [38]:
# ------------------------------------------------------------
# 4) MONTA TABELA POR REGIÃO + p-VALUES
# ------------------------------------------------------------
# Garante que todas as categorias existam em todas as regiões (preenche 0)
all_cats = sorted(num_br['SVN_cat'].unique().tolist())
all_regs = sorted(denom_reg['nasc_REGIAO'].unique().tolist())
grid = pd.MultiIndex.from_product([all_regs, all_cats],
                                  names=['nasc_REGIAO','SVN_cat']).to_frame(index=False)

num_reg_full = grid.merge(num_reg, on=['nasc_REGIAO','SVN_cat'], how='left')
num_reg_full['n_cat_reg'] = num_reg_full['n_cat_reg'].fillna(0).astype(int)
num_reg_full = num_reg_full.merge(denom_reg, on='nasc_REGIAO', how='left')

# junta p_br por categoria
num_reg_full = num_reg_full.merge(num_br[['SVN_cat','p_br']], on='SVN_cat', how='left')

# calcula % (IC) e p-value
num_reg_full['percent_ic'] = num_reg_full.apply(
    lambda r: prev_ci_wald(r['n_cat_reg'], r['n_total_reg']), axis=1
)
num_reg_full['p_value'] = num_reg_full.apply(
    lambda r: pvalue_reg_vs_br(r['n_cat_reg'], r['n_total_reg'], r['p_br']), axis=1
)

# formata p-value
def fmt_p(p):
    if pd.isna(p): 
        return ''
    if p < 0.001:
        return '<0.001'
    return f'{p:.3f}'

num_reg_full['p_value_fmt'] = num_reg_full['p_value'].apply(fmt_p)


In [40]:
# ------------------------------------------------------------
# 5) REORGANIZA NO LAYOUT DA SUA TABELA
#    (3 blocos com p-value + 1 bloco final sem p-value para Term non-SGA)
# ------------------------------------------------------------
def is_term_non_sga(cat_name):
    return cat_name.lower() in ['term and non-sga', 'term & non-sga', 'term non-sga']

wide_parts = []
for cat in all_cats:
    sub = (num_reg_full[num_reg_full['SVN_cat'] == cat]
           .loc[:, ['nasc_REGIAO','percent_ic','p_value_fmt']]
           .rename(columns={'nasc_REGIAO':'Region',
                            'percent_ic': f'{cat} % (95% CI)',
                            'p_value_fmt': f'{cat} p-value'}))
    # se for Term and non-SGA, deixa p-value vazio
    if is_term_non_sga(cat):
        sub[f'{cat} p-value'] = ''
    wide_parts.append(sub)

tabela_reg = wide_parts[0]
for part in wide_parts[1:]:
    tabela_reg = tabela_reg.merge(part, on='Region', how='left')

# adiciona N total por região
tabela_reg = tabela_reg.merge(denom_reg.rename(columns={'nasc_REGIAO':'Region',
                                                        'n_total_reg':'N nascidos vivos (região)'}),
                              on='Region', how='left')

# ordena colunas (cat ordem = all_cats)
col_order = ['Region']
for cat in all_cats:
    col_order += [f'{cat} % (95% CI)', f'{cat} p-value']
col_order += ['N nascidos vivos (região)']
tabela_reg = tabela_reg[col_order]

In [42]:
# ------------------------------------------------------------
# 6) LINHA DO BRASIL
# ------------------------------------------------------------
linha_br = {'Region': 'Brasil',
            'N nascidos vivos (região)': int(n_total_br)}
for cat in all_cats:
    n_cat_br = int(num_br.loc[num_br['SVN_cat'] == cat, 'n_cat_br'])
    linha_br[f'{cat} % (95% CI)'] = prev_ci_wald(n_cat_br, n_total_br)
    linha_br[f'{cat} p-value'] = ''  # referência nacional

tabela_final = pd.concat([tabela_reg, pd.DataFrame([linha_br])], ignore_index=True)

C:\Users\gabri\AppData\Local\Temp\ipykernel_8972\3429176508.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  n_cat_br = int(num_br.loc[num_br['SVN_cat'] == cat, 'n_cat_br'])


In [44]:
# ------------------------------------------------------------
# 7) RESULTADO
# ------------------------------------------------------------
print("\nTabela de Prevalência por 100 NV (% e IC95%) e p-valor (Região vs Brasil):")
print(tabela_final)


Tabela de Prevalência por 100 NV (% e IC95%) e p-valor (Região vs Brasil):
         Region Preterm and SGA % (95% CI) Preterm and SGA p-value  \
0  Centro-Oeste           2.15 (2.13-2.17)                  <0.001   
1      Nordeste           2.03 (2.02-2.04)                  <0.001   
2         Norte           1.76 (1.75-1.78)                  <0.001   
3       Sudeste           2.59 (2.58-2.60)                  <0.001   
4           Sul           2.41 (2.40-2.43)                  <0.001   
5        Brasil           2.29 (2.28-2.29)                           

  Preterm non-SGA % (95% CI) Preterm non-SGA p-value Term and SGA % (95% CI)  \
0           8.94 (8.90-8.98)                  <0.001     15.09 (15.04-15.13)   
1           9.42 (9.39-9.44)                  <0.001     15.47 (15.45-15.50)   
2        10.46 (10.43-10.50)                  <0.001     16.21 (16.17-16.26)   
3           8.65 (8.63-8.66)                  <0.001     15.21 (15.19-15.23)   
4           8.67 (8.64-8.70)     

In [46]:
tabela_final

,Region,Preterm and SGA % (95% CI),Preterm and SGA p-value,Preterm non-SGA % (95% CI),Preterm non-SGA p-value,Term and SGA % (95% CI),Term and SGA p-value,Term and non-SGA % (95% CI),Term and non-SGA p-value,N nascidos vivos (região)
0,Centro-Oeste,2.15 (2.13-2.17),<0.001,8.94 (8.90-8.98),<0.001,15.09 (15.04-15.13),0.102,73.82 (73.76-73.88),,2347474
1,Nordeste,2.03 (2.02-2.04),<0.001,9.42 (9.39-9.44),<0.001,15.47 (15.45-15.50),<0.001,73.08 (73.05-73.11),,7714169
2,Norte,1.76 (1.75-1.78),<0.001,10.46 (10.43-10.50),<0.001,16.21 (16.17-16.26),<0.001,71.56 (71.51-71.61),,2965287
3,Sudeste,2.59 (2.58-2.60),<0.001,8.65 (8.63-8.66),<0.001,15.21 (15.19-15.23),<0.001,73.55 (73.53-73.58),,10979625
4,Sul,2.41 (2.40-2.43),<0.001,8.67 (8.64-8.70),<0.001,13.37 (13.33-13.40),<0.001,75.55 (75.51-75.59),,3827104
5,Brasil,2.29 (2.28-2.29),,9.08 (9.07-9.09),,15.13 (15.11-15.14),,73.51 (73.49-73.52),,27833659
